In [1]:
! pip install scikit-learn-intelex


In [2]:
from sklearnex import patch_sklearn
# import logging
# logging.disable(logging.INFO)
patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
PATH="/kaggle/input/webis-clickbait-22/"
sep='[SEP]'

label_encoder = LabelEncoder()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:

def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        i['text']=i['postText']+sep+i['targetParagraphs']+sep+i['targetTitle']+sep+i['targetDescription']+sep+i['targetKeywords']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y


In [4]:

x_train,y_train = get_split_data(PATH+'train.jsonl')
x_test,y_test = get_split_data(PATH+'test.jsonl',False)
x_val,y_val = get_split_data(PATH+'validation.jsonl',False)


len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [5]:
# len(x_train),len(y_train),len(x_test),len(y_test),len(x_val),len(y_val),len(x_train[0]),len(y_train[0])
x_train.shape,y_train.shape,x_test.shape,y_test.shape,x_val.shape,y_val.shape,len(x_train[10])


((3200,), (3200,), (1000,), (1000,), (800,), (800,), 3518)

In [6]:
type(x_train),type(y_train),type(x_test),type(y_test),type(x_val),type(y_val),type(x_train[0]),type(y_train[0])

(numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.str_,
 numpy.int64)

In [7]:
param_grid = {
    "vectorizer__stop_words":[None,"english"],
    'vectorizer__ngram_range':[(1,1),(1,2),(2,2)],
    'model_rf__n_estimators': [100, 200, 300],
    'model_rf__max_depth': [3,5,10],
    'model_rf__min_samples_leaf': [1,2],
    'model_rf__max_features': ['sqrt', 'log2'],
    'model_rf__bootstrap': [True, False]
    }
# param_grid = {
#     "vectorizer__stop_words":[None],
#     'vectorizer__ngram_range':[(1,1)],
#     'model_rf__n_estimators': [100],
#     'model_rf__max_depth': [3],
#     'model_rf__min_samples_leaf': [2],
#     'model_rf__max_features': ['sqrt'],
#     'model_rf__bootstrap': [True]
#     }
tf_idf_pipeline = Pipeline([
    ('vectorizer',TfidfVectorizer()),
    ('model_rf',RandomForestClassifier(n_jobs=-1))
])
grid_search = GridSearchCV(tf_idf_pipeline, param_grid, cv=5, n_jobs=-1,verbose=10)
grid_search.fit(x_train, y_train)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits


INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:skl

[CV 1/5; 1/432] START model_rf__bootstrap=True, model_rf__max_depth=3, model_rf__max_features=sqrt, model_rf__min_samples_leaf=1, model_rf__n_estimators=100, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None
[CV 1/5; 1/432] END model_rf__bootstrap=True, model_rf__max_depth=3, model_rf__max_features=sqrt, model_rf__min_samples_leaf=1, model_rf__n_estimators=100, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.444 total time=   4.3s
[CV 2/5; 2/432] START model_rf__bootstrap=True, model_rf__max_depth=3, model_rf__max_features=sqrt, model_rf__min_samples_leaf=1, model_rf__n_estimators=100, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english
[CV 2/5; 2/432] END model_rf__bootstrap=True, model_rf__max_depth=3, model_rf__max_features=sqrt, model_rf__min_samples_leaf=1, model_rf__n_estimators=100, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.439 total time=   4.0s
[CV 5/5; 2/432] START model_rf__bootstrap=True, model_rf__ma

INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn


[CV 3/5; 1/432] START model_rf__bootstrap=True, model_rf__max_depth=3, model_rf__max_features=sqrt, model_rf__min_samples_leaf=1, model_rf__n_estimators=100, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None
[CV 3/5; 1/432] END model_rf__bootstrap=True, model_rf__max_depth=3, model_rf__max_features=sqrt, model_rf__min_samples_leaf=1, model_rf__n_estimators=100, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.442 total time=   4.4s
[CV 1/5; 2/432] START model_rf__bootstrap=True, model_rf__max_depth=3, model_rf__max_features=sqrt, model_rf__min_samples_leaf=1, model_rf__n_estimators=100, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english
[CV 1/5; 2/432] END model_rf__bootstrap=True, model_rf__max_depth=3, model_rf__max_features=sqrt, model_rf__min_samples_leaf=1, model_rf__n_estimators=100, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.434 total time=   3.9s
[CV 4/5; 2/432] START model_rf__bootstrap=True, model_rf__ma

...

[CV 1/5; 409/432] START model_rf__bootstrap=False, model_rf__max_depth=10, model_rf__max_features=log2, model_rf__min_samples_leaf=1, model_rf__n_estimators=300, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None
[CV 1/5; 409/432] END model_rf__bootstrap=False, model_rf__max_depth=10, model_rf__max_features=log2, model_rf__min_samples_leaf=1, model_rf__n_estimators=300, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.450 total time=   4.8s
[CV 5/5; 409/432] START model_rf__bootstrap=False, model_rf__max_depth=10, model_rf__max_features=log2, model_rf__min_samples_leaf=1, model_rf__n_estimators=300, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None
[CV 5/5; 409/432] END model_rf__bootstrap=False, model_rf__max_depth=10, model_rf__max_features=log2, model_rf__min_samples_leaf=1, model_rf__n_estimators=300, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.441 total time=   5.0s
[CV 4/5; 410/432] START model_rf__bootstrap=False,

INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn


[CV 2/5; 410/432] START model_rf__bootstrap=False, model_rf__max_depth=10, model_rf__max_features=log2, model_rf__min_samples_leaf=1, model_rf__n_estimators=300, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english
[CV 2/5; 410/432] END model_rf__bootstrap=False, model_rf__max_depth=10, model_rf__max_features=log2, model_rf__min_samples_leaf=1, model_rf__n_estimators=300, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.439 total time=   4.6s
[CV 5/5; 410/432] START model_rf__bootstrap=False, model_rf__max_depth=10, model_rf__max_features=log2, model_rf__min_samples_leaf=1, model_rf__n_estimators=300, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english
[CV 5/5; 410/432] END model_rf__bootstrap=False, model_rf__max_depth=10, model_rf__max_features=log2, model_rf__min_samples_leaf=1, model_rf__n_estimators=300, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.428 total time=   4.6s
[CV 4/5; 411/432] START model_rf__boot

INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:skl

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                                       ('model_rf',
                                        RandomForestClassifier(n_jobs=-1))]),
             n_jobs=-1,
             param_grid={'model_rf__bootstrap': [True, False],
                         'model_rf__max_depth': [3, 5, 10],
                         'model_rf__max_features': ['sqrt', 'log2'],
                         'model_rf__min_samples_leaf': [1, 2],
                         'model_rf__n_estimators': [100, 200, 300],
                         'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)],
                         'vectorizer__stop_words': [None, 'english']},
             verbose=10)

In [8]:
optimal_estimators = grid_search.best_params_["model_rf__n_estimators"]
optimal_depth=grid_search.best_params_["model_rf__max_depth"]
optimal_min_leaf=grid_search.best_params_['model_rf__min_samples_leaf']
optimal_max_features=grid_search.best_params_['model_rf__max_features']
optimal_bootstrap=grid_search.best_params_['model_rf__bootstrap']
optimal_ngram = grid_search.best_params_["vectorizer__ngram_range"]
optimal_stop_words = grid_search.best_params_["vectorizer__stop_words"]
print("optimal_estimators: ",optimal_estimators)
print("optimal_depth: ",optimal_depth)
print("optimal_min_leaf: ",optimal_min_leaf)
print("optimal_max_features: ",optimal_max_features)
print("optimal_bootstrap: ",optimal_bootstrap)
print("optimal_ngram: ",optimal_ngram)
print("optimal_stop_words: ",optimal_stop_words)
# print(grid_search.best_estimator_)
print("Best parameters found by GridSearchCV:")
print(grid_search.best_params_)



optimal_estimators:  200
optimal_depth:  10
optimal_min_leaf:  1
optimal_max_features:  sqrt
optimal_bootstrap:  False
optimal_ngram:  (1, 1)
optimal_stop_words:  None
Best parameters found by GridSearchCV:
{'model_rf__bootstrap': False, 'model_rf__max_depth': 10, 'model_rf__max_features': 'sqrt', 'model_rf__min_samples_leaf': 1, 'model_rf__n_estimators': 200, 'vectorizer__ngram_range': (1, 1), 'vectorizer__stop_words': None}


In [9]:

vectorizer = TfidfVectorizer(ngram_range=optimal_ngram,stop_words=optimal_stop_words)
x_train =  vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)
x_val = vectorizer.transform(x_val)
print(x_train.shape)
print(x_test.shape)



INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


(3200, 56343)
(1000, 56343)


In [10]:
rf_model=RandomForestClassifier(n_estimators=optimal_estimators,max_depth=optimal_depth,min_samples_leaf=optimal_min_leaf,max_features=optimal_max_features,bootstrap=optimal_bootstrap,n_jobs=-1)
rf_model.fit(x_train, y_train)
y_train_pred = rf_model.predict(x_train)
y_test_pred = rf_model.predict(x_test)
y_val_pred = rf_model.predict(x_val)
print("Training data metrics : ")
print(accuracy_score(y_train_pred,y_train))

print("Testing data accuracy")
print(accuracy_score(y_test,y_test_pred))

print("Validation data accuracy")
print(accuracy_score(y_val,y_val_pred))



INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:

Training data metrics : 
0.78875
Testing data accuracy
0.508
Validation data accuracy
0.49125


In [11]:
print("Training data metrics:")
print(balanced_accuracy_score(y_train_pred, y_train))
print(f1_score(y_train_pred, y_train, average='weighted'))
print(classification_report(y_train_pred, y_train))

print("Testing data metrics:")
print(balanced_accuracy_score(y_test_pred, y_test))
print(f1_score(y_test_pred, y_test, average='weighted'))
print(classification_report(y_test_pred, y_test))

print("Validation data metrics:")
print(balanced_accuracy_score(y_val_pred, y_val))
print(f1_score(y_val_pred, y_val, average='weighted'))
print(classification_report(y_val_pred, y_val))

Training data metrics:
0.8766706989247312
0.8020990054888002
              precision    recall  f1-score   support

           0       0.39      1.00      0.56       216
           1       0.74      0.94      0.83      1000
           2       1.00      0.69      0.82      1984

    accuracy                           0.79      3200
   macro avg       0.71      0.88      0.73      3200
weighted avg       0.88      0.79      0.80      3200

Testing data metrics:
0.5930952218563755
0.5566517240123232
              precision    recall  f1-score   support

           0       0.06      0.77      0.11        13
           1       0.40      0.51      0.45       316
           2       0.80      0.50      0.62       671

    accuracy                           0.51      1000
   macro avg       0.42      0.59      0.39      1000
weighted avg       0.66      0.51      0.56      1000

Validation data metrics:
0.5246176752988325
0.5445442234488008
              precision    recall  f1-score   support


In [12]:
pickle.dump((rf_model,label_encoder,vectorizer),open('model_rf.pkl','wb'))

In [13]:
model2,encoder,vectorizer = pickle.load(open("model_rf.pkl","rb"))

In [14]:
y_train_pred = model2.predict(x_train)
y_test_pred = model2.predict(x_test)
y_val_pred = model2.predict(x_val)
print("Training data metrics : ")
print(accuracy_score(y_train_pred,y_train))

print("Testing data accuracy")
print(accuracy_score(y_test,y_test_pred))

print("Testing data accuracy")
print(accuracy_score(y_val,y_val_pred))

INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.ensemble.RandomForestClassifier.predict_proba: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Training data metrics : 
0.78875
Testing data accuracy
0.508
Testing data accuracy
0.49125


In [15]:
print("Training data metrics:")
print(balanced_accuracy_score(y_train_pred, y_train))
print(f1_score(y_train_pred, y_train, average='weighted'))
print(classification_report(y_train_pred, y_train))

print("Testing data metrics:")
print(balanced_accuracy_score(y_test_pred, y_test))
print(f1_score(y_test_pred, y_test, average='weighted'))
print(classification_report(y_test_pred, y_test))

print("Validation data metrics:")
print(balanced_accuracy_score(y_val_pred, y_val))
print(f1_score(y_val_pred, y_val, average='weighted'))
print(classification_report(y_val_pred, y_val))

Training data metrics:
0.8766706989247312
0.8020990054888002
              precision    recall  f1-score   support

           0       0.39      1.00      0.56       216
           1       0.74      0.94      0.83      1000
           2       1.00      0.69      0.82      1984

    accuracy                           0.79      3200
   macro avg       0.71      0.88      0.73      3200
weighted avg       0.88      0.79      0.80      3200

Testing data metrics:
0.5930952218563755
0.5566517240123232
              precision    recall  f1-score   support

           0       0.06      0.77      0.11        13
           1       0.40      0.51      0.45       316
           2       0.80      0.50      0.62       671

    accuracy                           0.51      1000
   macro avg       0.42      0.59      0.39      1000
weighted avg       0.66      0.51      0.56      1000

Validation data metrics:
0.5246176752988325
0.5445442234488008
              precision    recall  f1-score   support
